In [2]:
import ollama
response = ollama.chat(model='llama3.1', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue to us because of a phenomenon called Rayleigh scattering, which occurs when sunlight interacts with the tiny molecules of gases in the Earth's atmosphere.

Here's what happens:

1. **Sunlight enters the atmosphere**: When the sun shines, it emits all kinds of electromagnetic radiation, including visible light, ultraviolet (UV) radiation, and other forms of energy.
2. **Light scatters off gas molecules**: As sunlight travels through the atmosphere, it encounters tiny molecules of gases like nitrogen (N2), oxygen (O2), and others. These molecules are much smaller than the wavelength of light, so they scatter the light in all directions.
3. **Blue light is scattered more**: The shorter wavelengths of light, like blue and violet, are scattered more efficiently than longer wavelengths, like red and orange. This is because the smaller gas molecules are more effective at scattering the shorter wavelengths.
4. **Our eyes perceive the scattered light**: When we look up at t

In [3]:
import pandas as pd
from phi.agent import Agent
from phi.model.groq import Groq

#import occupation description
df_occupation = pd.read_csv('occupation.txt', sep='\t')
df_occupation = df_occupation.dropna(subset=["occupation_description"])
df_occupation = df_occupation[["occupation_code", "occupation_name","occupation_description"]]
df_occupation = df_occupation.rename(columns={"occupation_code": "OCC_CODE"})
#print(df_occupation.head())

#import wage stat
df_wage = pd.read_excel("testocc.xlsx")
df_wage = df_wage[["OCC_CODE", "A_MEAN", "O_GROUP"]]
df_wage['OCC_CODE'] = df_wage['OCC_CODE'].str.replace('-', '').astype(int)
df_wage = df_wage[df_wage['O_GROUP'] == "detailed"]
#print(df_wage.head())

#join both dfs together
merged_df = pd.merge(df_occupation, df_wage, on='OCC_CODE', how='inner').drop(columns=["O_GROUP"])
print(merged_df.head())

#testing the idea
test_df = merged_df.head(5)

   OCC_CODE                      occupation_name  \
0    111011                     Chief Executives   
1    111021      General and Operations Managers   
2    111031                          Legislators   
3    112011  Advertising and Promotions Managers   
4    112021                   Marketing Managers   

                              occupation_description  A_MEAN  
0  Determine and formulate policies and provide o...  258900  
1  Plan, direct, or coordinate the operations of ...  129330  
2  Develop, introduce, or enact laws and statutes...   68140  
3  Plan, direct, or coordinate advertising polici...  152620  
4  Plan, direct, or coordinate marketing policies...  166410  


In [3]:
import ollama
import pandas as pd

response_v = ollama.chat(
    model='llama3.2-vision',
    messages=[{
        'role': 'user',
        'content': 'describe what you see',
        'images': ['test.png']
    }]
)

response_c = ollama.chat(
    model='llama3.2:latest',
    messages=[{
        'role': 'user',
        'content': response_v['message']["content"] + "\n please base on the above information, produce a pandas dataframme to contain the information. no need to greet or make conclusion, directly output your code. assume this structure: data = { [your code]} \n df = pd.DataFrame(data)",        
    }]
)
data = response_c["message"]["content"]
df = pd.DataFrame(data)

df.head()

KeyboardInterrupt: 

In [2]:
pip install ollama

  Using cached ollama-0.3.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.6-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached ollama-0.3.3-py3-none-any.whl (10 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
Using cached httpcore-1.0.6-py3-none-any.whl (78 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
import pandas as pd

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=BTC&to_currency=EUR&apikey=OYG3LC7YO5WNTKED'
r = requests.get(url)
df = pd.DataFrame(r.json())

df

,Realtime Currency Exchange Rate
1. From_Currency Code,BTC
2. From_Currency Name,Bitcoin
3. To_Currency Code,EUR
4. To_Currency Name,Euro
5. Exchange Rate,85444.09000000
6. Last Refreshed,2024-11-13 14:51:03
7. Time Zone,UTC
8. Bid Price,85441.01800000
9. Ask Price,85448.00700000
